In [ ]:
# HealthBuddy — AI Multi-Agent Healthcare Assistant
### OCR + Parser + Medical Summary + Symptom Triage + Personalized Health Tips


In [11]:
!apt-get install poppler-utils -y
!pip install pytesseract pdf2image Pillow rapidfuzz python-dotenv


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.12).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [12]:
import os

folders = ["agents", "tools", "memory", "data"]
for f in folders:
    os.makedirs(f, exist_ok=True)
folders


['agents', 'tools', 'memory', 'data']

In [13]:
from google.colab import files
uploaded = files.upload()

filepath = next(iter(uploaded))
filepath


Saving 1800L20259080035.pdf to 1800L20259080035 (1).pdf


'1800L20259080035 (1).pdf'

In [14]:
%%writefile tools/ocr_tool.py
import pytesseract
from pdf2image import convert_from_path
from PIL import Image
import os

class OCRTool:
    def __init__(self):
        pass

    def run(self, file_path):
        text = ""

        # If PDF
        if file_path.endswith(".pdf"):
            pages = convert_from_path(file_path)
            for page in pages:
                text += pytesseract.image_to_string(page)

        # If Image (JPG/PNG)
        else:
            img = Image.open(file_path)
            text = pytesseract.image_to_string(img)

        return text

ocr_tool = OCRTool()


Overwriting tools/ocr_tool.py


In [15]:
from tools.ocr_tool import ocr_tool

text = ocr_tool.run(filepath)
print(text[:500])


Pathkindll>

 

 

Mrs. SEMA KHATOON Collected at: Mankind Pharma Limited Collected : 08/09/2025 03:43 PM
Diacar D 205 2nd Fir Tower Il Seawood

Age : 30 Yrs Grand Central Navi Mumbai 400706 Ph Reported =: 08/09/2025 04:35 PM

Gender : Female 9999999999 Report Status : Final

PID : 1800250908007414 .

VID - 1800L20259080035 Processed at: Pathkind Labs Patna, Ref. By : DR ISHTIAQUE

. Aryans 1st Floor South Bypass Rd East AHMAD

Jaganpura Khemnichak 800027, Ph
7827949761

 

Barcode: 80013580737



In [16]:
%%writefile tools/ocr_tool.py


Overwriting tools/ocr_tool.py


In [17]:
%%writefile tools/ocr_tool.py
import pytesseract
from pdf2image import convert_from_path
from PIL import Image

class OCRTool:
    def __init__(self):
        pass

    def run(self, file_path):
        text = ""

        # If PDF
        if file_path.endswith(".pdf"):
            pages = convert_from_path(file_path)
            for page in pages:
                text += pytesseract.image_to_string(page)

        # If image (jpg/png)
        else:
            img = Image.open(file_path)
            text = pytesseract.image_to_string(img)

        return text

ocr_tool = OCRTool()


Overwriting tools/ocr_tool.py


In [18]:
import importlib
from tools import ocr_tool
importlib.reload(ocr_tool)


<module 'tools.ocr_tool' from '/content/tools/ocr_tool.py'>

In [19]:
%%writefile tools/parser_tool.py
import re

class ParserTool:
    def __init__(self):
        self.tests = {
            "hemoglobin": ["hemoglobin", "hb"],
            "wbc": ["wbc", "white blood cell", "total leukocyte"],
            "platelets": ["platelet", "plt"],
            "bilirubin_total": ["bilirubin"],
            "sgot": ["sgot", "ast"],
            "sgpt": ["sgpt", "alt"],
            "creatinine": ["creatinine"],
            "urea": ["urea"],
            "tsh": ["tsh"],
            "t3": ["t3"],
            "t4": ["t4"],
            "vitamin_d": ["vit d", "vitamin d", "25-oh"],
            "vitamin_b12": ["b12", "vitamin b12"],
            "fbs": ["fasting", "fbs"],
            "ppbs": ["pp", "post prandial"],
            "hba1c": ["hba1c"],
            "fsh": ["fsh"],
            "lh": ["lh"],
            "prolactin": ["prolactin"],
            "amh": ["amh", "anti mullerian"],
        }

    def detect_test(self, line_lower):
        for test, keys in self.tests.items():
            if any(k in line_lower for k in keys):
                return test
        return None

    def extract_numbers(self, line):
        return re.findall(r"[0-9]+\.?[0-9]*", line)

    def detect_unit(self, line_lower):
        m = re.search(r"(g/dl|mg/dl|ng/ml|iu/l|mlu/ml|%|/ul)", line_lower)
        return m.group(1) if m else ""

    def parse_line(self, line):
        line_lower = line.lower()
        test = self.detect_test(line_lower)
        if not test:
            return None, None

        nums = self.extract_numbers(line)
        if len(nums) == 0:
            return None, None

        value = float(nums[0])
        ref_range = None
        if len(nums) >= 3:
            ref_range = f"{nums[1]} - {nums[2]}"

        return test, {
            "value": value,
            "unit": self.detect_unit(line_lower),
            "flag": "Low" if "low" in line_lower else ("High" if "high" in line_lower else ""),
            "reference_range": ref_range
        }

    def parse(self, text):
        structured = {}
        for line in text.split("\n"):
            test, info = self.parse_line(line)
            if test and info:
                structured[test] = info
        return structured

parser_tool = ParserTool()


Overwriting tools/parser_tool.py


In [20]:
import importlib
from tools import parser_tool
importlib.reload(parser_tool)


<module 'tools.parser_tool' from '/content/tools/parser_tool.py'>

In [21]:
%%writefile agents/ingest_agent.py
import json
import os
from tools.ocr_tool import ocr_tool
from tools.parser_tool import parser_tool

class IngestAgent:
    def __init__(self, memory_dir="memory"):
        self.memory_dir = memory_dir
        os.makedirs(self.memory_dir, exist_ok=True)

    def run(self, file_path, patient_id="default"):
        text = ocr_tool.run(file_path)
        data = parser_tool.parse(text)

        # Save to memory
        path = os.path.join(self.memory_dir, f"{patient_id}.json")
        with open(path, "w") as f:
            json.dump(data, f, indent=4)

        return data


Overwriting agents/ingest_agent.py


In [22]:
%%writefile agents/summary_agent.py
import json, os

def gemini_generate(prompt):
    return "Summary:\n" + prompt[:400]

class SummaryAgent:
    def __init__(self, memory_dir="memory"):
        self.memory_dir = memory_dir

    def run(self, patient_id="default"):
        path = os.path.join(self.memory_dir, f"{patient_id}.json")
        if not os.path.exists(path):
            return {"error": "No data found"}

        data = json.load(open(path))
        prompt = str(data)
        return {"summary": gemini_generate(prompt)}


Overwriting agents/summary_agent.py


In [23]:
%%writefile agents/symptom_agent.py
class SymptomAgent:
    def simple_rule_based_triage(self, symptoms):
        s = symptoms.lower()
        if any(x in s for x in ["severe","chest pain","unconscious","difficulty breathing"]):
            return "High"
        if any(x in s for x in ["fever","dizziness","fatigue","vomiting"]):
            return "Moderate"
        return "Low"

    def run(self, symptoms):
        risk = self.simple_rule_based_triage(symptoms)
        return {
            "risk": risk,
            "english": "Your symptoms are " + risk,
            "hindi": "आपके लक्षण " + risk + " हैं"
        }


Overwriting agents/symptom_agent.py


In [24]:
%%writefile agents/tips_agent.py
import json, os

class TipsAgent:
    def run(self, patient_id="default"):
        path = os.path.join("memory", f"{patient_id}.json")
        data = json.load(open(path))

        return {
            "tips": [
                "Drink water",
                "Walk daily",
                "Sleep well"
            ]
        }


Overwriting agents/tips_agent.py


In [25]:
%%writefile agents/orchestrator.py
from agents.ingest_agent import IngestAgent
from agents.summary_agent import SummaryAgent
from agents.symptom_agent import SymptomAgent
from agents.tips_agent import TipsAgent

class HealthAgentOrchestrator:
    def __init__(self):
        self.ingest = IngestAgent()
        self.summary = SummaryAgent()
        self.symptom = SymptomAgent()
        self.tips = TipsAgent()

    def run(self, action, **kwargs):
        if action == "upload_report":
            return self.ingest.run(kwargs["file_path"], kwargs.get("patient_id","user1"))
        if action == "summary":
            return self.summary.run(kwargs.get("patient_id","user1"))
        if action == "symptoms":
            return self.symptom.run(kwargs["text"])
        if action == "tips":
            return self.tips.run(kwargs.get("patient_id","user1"))
        return {"error": "Invalid action"}


Overwriting agents/orchestrator.py


In [26]:
from agents.orchestrator import HealthAgentOrchestrator

orc = HealthAgentOrchestrator()

def print_section(title):
    print("\n" + "="*60)
    print(title)
    print("="*60 + "\n")


def pretty_print_tests(test_dict):
    for test, info in test_dict.items():
        test_name = test.upper().replace("_", " ")

        value = info.get("value", "")
        unit = info.get("unit", "")
        flag = info.get("flag", "")
        ref = info.get("reference_range", "")

        line = f"{test_name}: {value} {unit}"

        if ref:
            line += f"  |  Reference Range: {ref}"
        if flag:
            line += f"  |  Flag: {flag}"

        print(line)
    print("\n")


# ----------------------------
# 1) Report upload
# ----------------------------
print_section("1) REPORT UPLOAD & PARSE RESULT")
parsed = orc.run("upload_report", file_path=filepath, patient_id="user1")
pretty_print_tests(parsed)


# ----------------------------
# 2) Summary
# ----------------------------
print_section("2) MEDICAL SUMMARY (English + Hindi)")
summary = orc.run("summary", patient_id="user1")

text = summary["summary"]

# Multi-line clean print
for line in text.split("\n"):
    print(line)

print("\n")



# ----------------------------
# 3) Symptoms
# ----------------------------
print_section("3) SYMPTOM TRIAGE & ADVICE")
sym = orc.run("symptoms", text="fatigue and dizziness since last night")

print("Risk Level:", sym.get("risk", "N/A"))
print("\nEnglish Advice:\n", sym.get("english", "N/A"))
print("\nHindi Advice:\n", sym.get("hindi", "N/A"), "\n")


# ----------------------------
# 4) Tips & reminders
# ----------------------------
print_section("4) PERSONALIZED HEALTH TIPS & REMINDERS")

tips = orc.run("tips", patient_id="user1")

# Always safe access
health_tips = tips.get("health_tips", {})
simple_tips = tips.get("tips", [])
reminders = tips.get("reminders", [])

# Case A: health_tips exists
if isinstance(health_tips, dict) and "english" in health_tips:
    print("Health Tips:")
    for t in health_tips["english"]:
        print(" -", t)

# Case B: simple tips list exists
elif isinstance(simple_tips, list):
    print("Health Tips:")
    for t in simple_tips:
        print(" -", t)

# Reminders (if exist)
if reminders:
    print("\nReminders:")
    for r in reminders:
        print(" -", r)

print("\n" + "-"*60)
print(" ALL AGENTS COMPLETED SUCCESSFULLY ")
print("-"*60)


1) REPORT UPLOAD & PARSE RESULT

SGOT: 1.0 
T3: 3.0 
T4: 4.0 
TSH: 3.0 
PPBS: 0.0 



2) MEDICAL SUMMARY (English + Hindi)

Summary:
{'sgot': {'value': 1.0, 'unit': '', 'flag': '', 'reference_range': None}, 't3': {'value': 3.0, 'unit': '', 'flag': '', 'reference_range': None}, 't4': {'value': 4.0, 'unit': '', 'flag': '', 'reference_range': None}, 'tsh': {'value': 3.0, 'unit': '', 'flag': '', 'reference_range': None}, 'ppbs': {'value': 0.0, 'unit': '', 'flag': '', 'reference_range': None}}



3) SYMPTOM TRIAGE & ADVICE

Risk Level: Moderate

English Advice:
 Your symptoms are Moderate

Hindi Advice:
 आपके लक्षण Moderate हैं 


4) PERSONALIZED HEALTH TIPS & REMINDERS

Health Tips:
 - Drink water
 - Walk daily
 - Sleep well

------------------------------------------------------------
 ALL AGENTS COMPLETED SUCCESSFULLY 
------------------------------------------------------------
